In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("hw8.ipynb")

In [ ]:
rng_seed = 50

In [ ]:
#imports
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import scipy as sp
import pandas as pd
#below line allows matplotlib plots to appear in cell output
%matplotlib inline

## **Question 1**: Gram-Schmidt Orthonormalization

The **Gram-Schmidt process** is a fundamental algorithm in linear algebra that takes a set of linearly independent vectors and produces an orthonormal set of vectors that spans the same subspace.

### The Gram-Schmidt Process

Given a set of linearly independent vectors $\{\mathbf{v}_1, \mathbf{v}_2, ..., \mathbf{v}_n\}$, the Gram-Schmidt process produces an orthonormal set $\{\mathbf{u}_1, \mathbf{u}_2, ..., \mathbf{u}_n\}$ through the following algorithm:

**Step 1: Orthogonalization**

For the first vector, simply normalize it:
$$\mathbf{u}_1 = \frac{\mathbf{v}_1}{||\mathbf{v}_1||}$$

For each subsequent vector $\mathbf{v}_k$, subtract off its projections onto all previously computed orthonormal vectors:

$$\mathbf{w}_k = \mathbf{v}_k - \sum_{i=1}^{k-1} (\mathbf{v}_k \cdot \mathbf{u}_i) \mathbf{u}_i$$

**Step 2: Normalization**

Then normalize the result to get the next orthonormal vector:
$$\mathbf{u}_k = \frac{\mathbf{w}_k}{||\mathbf{w}_k||}$$

The key insight is that at each step, we remove all components of $\mathbf{v}_k$ that lie in the span of the previous vectors, leaving only the component perpendicular to that subspace.

### Linear Independence Check

Before applying Gram-Schmidt, we must verify that the input vectors are linearly independent. A set of vectors is linearly independent if and only if the matrix formed by these vectors (as columns) has full rank. If the vectors are linearly dependent, the Gram-Schmidt process cannot proceed because at some step $k$, the vector $\mathbf{w}_k$ will be the zero vector (or numerically very close to zero).

Write a function `gram_schmidt(vectors)` that implements the Gram-Schmidt orthonormalization process.

**Requirements:**
- Input `vectors` is a 2D numpy array where each **row** is a vector
- First check if the vectors are linearly independent using matrix rank
- If they are linearly dependent, return `None`
- If they are linearly independent, apply the Gram-Schmidt process
- Return a 2D numpy array where each **row** is an orthonormal vector
- The returned vectors should satisfy:
  - Each vector has unit length (norm = 1)
  - All vectors are mutually orthogonal (dot product = 0)

**Parameters:**
- `vectors`: 2D numpy array of shape (n, d) where n is the number of vectors and d is the dimension

**Returns:**
- `orthonormal_vectors`: 2D numpy array of shape (n, d) containing orthonormal vectors, or `None` if input vectors are linearly dependent

**Hints:**
- Use `np.linalg.matrix_rank()` to check linear independence
- Use `np.linalg.norm()` to compute vector norms
- Use `np.dot()` for dot products
- Be careful with numerical precision - use a small tolerance (e.g., 1e-10) when checking for zero vectors

In [ ]:
def gram_schmidt(vectors):
    # Convert to float array
    vectors = np.array(vectors, dtype=float)
    n, d = vectors.shape
    
    # Check linear independence
    ...
    
    # Initialize output array
    orthonormal_vectors = np.zeros((n, d))
    
    # Apply Gram-Schmidt process
    ...
    
    return orthonormal_vectors

In [ ]:
grader.check("q1")

## **Question 2**: Fourier Series Fitting

Fourier series provide a powerful method for representing periodic functions as sums of sine and cosine functions. Any periodic function $f(t)$ with period $T = 2\pi$ can be approximated by a Fourier series:

$$f(t) \approx a_0 + \sum_{k=1}^{K} \left[ a_k \cos(kt) + b_k \sin(kt) \right]$$

where:
- $a_0$ is the constant (DC) term
- $a_k$ are the cosine coefficients
- $b_k$ are the sine coefficients
- $K$ is the highest Fourier mode (frequency) we include

This is a **linear problem** because $f(t)$ depends linearly on the coefficients $\{a_0, a_1, b_1, a_2, b_2, ..., a_K, b_K\}$.

### Least Squares Formulation

Given $N$ data points $(t_i, y_i)$ for $i = 1, 2, ..., N$, we want to find the coefficients that minimize the sum of squared errors. This can be written as a linear system:

$$\mathbf{A} \mathbf{x} = \mathbf{y}$$

where:
- $\mathbf{y}$ is the data vector: $\mathbf{y} = [y_1, y_2, ..., y_N]^T$
- $\mathbf{x}$ is the coefficient vector: $\mathbf{x} = [a_0, a_1, b_1, a_2, b_2, ..., a_K, b_K]^T$ (has length $2K+1$)
- $\mathbf{A}$ is the design matrix where each row corresponds to a data point:

$$\mathbf{A} = \begin{bmatrix}
1 & \cos(t_1) & \sin(t_1) & \cos(2t_1) & \sin(2t_1) & \cdots & \cos(Kt_1) & \sin(Kt_1) \\
1 & \cos(t_2) & \sin(t_2) & \cos(2t_2) & \sin(2t_2) & \cdots & \cos(Kt_2) & \sin(Kt_2) \\
\vdots & \vdots & \vdots & \vdots & \vdots & \ddots & \vdots & \vdots \\
1 & \cos(t_N) & \sin(t_N) & \cos(2t_N) & \sin(2t_N) & \cdots & \cos(Kt_N) & \sin(Kt_N)
\end{bmatrix}$$

The least squares solution is given by solving the **normal equations**:

$$\mathbf{A}^T \mathbf{A} \mathbf{x} = \mathbf{A}^T \mathbf{y}$$

or equivalently, using numpy's least squares solver: $\mathbf{x} = (\mathbf{A}^T \mathbf{A})^{-1} \mathbf{A}^T \mathbf{y}$

### Part a: Constructing the Design Matrix

Write a function `create_fourier_design_matrix(filename, K)` that reads data from a CSV file and constructs the design matrix and data vector for Fourier fitting.

**Requirements:**
- Read the CSV file using pandas (columns: 'time', 'amplitude')
- Extract the time values $t_i$ and amplitude values $y_i$
- Construct the design matrix $\mathbf{A}$ with shape $(N, 2K+1)$ where:
  - Column 0: all ones (for $a_0$)
  - Columns $1, 2, 3, 4, ..., 2K-1, 2K$: alternating $\cos(kt)$ and $\sin(kt)$ for $k=1,2,...,K$
- Return both the design matrix and the data vector

**Parameters:**
- `filename`: str, path to CSV file with 'time' and 'amplitude' columns
- `K`: int, highest Fourier mode to include

**Returns:**
- `(A, y)`: tuple of numpy arrays
  - `A`: design matrix of shape (N, 2K+1)
  - `y`: data vector of shape (N,)

**Hints:**
- Use `pd.read_csv()` to load the data
- Use `np.cos()` and `np.sin()` to compute trigonometric functions
- Build the matrix column by column or use array broadcasting

In [ ]:
def create_fourier_design_matrix(filename, K):
    
    # Read the data
    data = pd.read_csv(filename)
    ...
    
    
    # Initialize design matrix
    
    # First column: constant term
    A[:, 0] = ...
    
    # Fill in cosine and sine columns
    
    
    return A, y

In [ ]:
grader.check("q2a")

### Part b: Solving for Fourier Coefficients and Plotting

Write a function `fit_fourier_series(filename, K, show_plot=False)` that:
1. Uses your function from part (a) to create the design matrix and data vector
2. Solves the least squares problem to find the best-fit Fourier coefficients
3. Creates a plot showing both the original data and the fitted curve
4. Returns the coefficients and the figure object

**Solving the Least Squares Problem:**

Given the design matrix $\mathbf{A}$ and data vector $\mathbf{y}$, we want to find the coefficient vector $\mathbf{x}$ that minimizes $||\mathbf{A}\mathbf{x} - \mathbf{y}||^2$.

The solution is given by the **normal equations**:
$$\mathbf{A}^T \mathbf{A} \mathbf{x} = \mathbf{A}^T \mathbf{y}$$

You can solve this using:
- **Option 1**: `np.linalg.lstsq(A, y, rcond=None)` - returns the least squares solution directly
- **Option 2**: Solve the normal equations: `x = np.linalg.solve(A.T @ A, A.T @ y)`

Both methods give the same result, but `lstsq` is more numerically stable.

**Plotting Requirements:**
- Plot the original data points as scatter plot
- Generate a smooth fitted curve by evaluating the Fourier series at many points
- Include appropriate labels, title, legend, and grid
- The function should return both the coefficients and the figure object

**Parameters:**
- `filename`: str, path to CSV file with data
- `K`: int, highest Fourier mode to include  
- `show_plot`: bool, default False. If True, call `plt.show()`

**Returns:**
- `(coefficients, fig)`: tuple containing:
  - `coefficients`: numpy array of shape (2K+1,) with fitted coefficients $[a_0, a_1, b_1, ..., a_K, b_K]$
  - `fig`: matplotlib figure object

**Hints:**
- Use your `create_fourier_design_matrix` function from part (a)
- To create the smooth fitted curve, generate many points (e.g., 500) evenly spaced over $[0, 2\pi]$
- Evaluate the Fourier series: $f(t) = a_0 + \sum_{k=1}^{K} [a_k \cos(kt) + b_k \sin(kt)]$

In [ ]:
def fit_fourier_series(filename, K, show_plot=False):
    
    # Get design matrix and data vector
    
    # Solve least squares problem
    coefficients = ...
    
    # Read original data for plotting
    
    
    # Evaluate Fourier series
    
    # Create plot
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Plot data and fit
    ...
    
    if show_plot:
        plt.show()
    
    return coefficients, fig

In [ ]:
# Test the Fourier fitting function
print("Testing Fourier series fitting with different K values:\n")

# Test with K=1
print("=" * 60)
print("K = 1 (fitting up to 1st harmonic)")
print("=" * 60)
coeffs_1, fig_1 = fit_fourier_series('fourier_data.csv', K=1, show_plot=True)
print(f"Coefficients: {coeffs_1}")
print(f"a_0 = {coeffs_1[0]:.4f}")
print(f"a_1 = {coeffs_1[1]:.4f}, b_1 = {coeffs_1[2]:.4f}")

# Test with K=5
print("=" * 60)
print("K = 5 (fitting up to 5th harmonic)")
print("=" * 60)
coeffs_5, fig_5 = fit_fourier_series('fourier_data.csv', K=5, show_plot=True)
print(f"Number of coefficients: {len(coeffs_5)}")
print(f"a_0 = {coeffs_5[0]:.4f}")
for k in range(1, 6):
    print(f"a_{k} = {coeffs_5[2*k-1]:.4f}, b_{k} = {coeffs_5[2*k]:.4f}")

print("\n" + "=" * 60)
print("Note: True signal was generated with:")
print("f(t) = 1.0 + 0.8*cos(t) + 0.5*sin(t) + 0.3*cos(2t) + 0.2*sin(2t)")
print("Compare the fitted coefficients to these true values!")
print("=" * 60)

In [ ]:
grader.check("q2b")

## **Question 3**: Weighted Least Squares Fitting

In many physics experiments, different data points have different levels of uncertainty. For example, some measurements might be taken with more precise instruments, or under better experimental conditions. **Weighted least squares** allows us to account for these varying uncertainties by giving more weight to more precise measurements.

### Physics Application: Ohm's Law

We will fit data to **Ohm's Law**, which states that the voltage $V$ across a resistor is proportional to the current $I$ flowing through it:

$$V = R \cdot I$$

where $R$ is the resistance (in Ohms). This is a linear relationship that passes through the origin.

### Weighted Least Squares Theory

When each data point $i$ has a measurement $(x_i, y_i)$ with uncertainty $\sigma_i$, we want to minimize the **weighted sum of squared residuals**:

$$\chi^2 = \sum_{i=1}^{N} \frac{(y_i - f(x_i))^2}{\sigma_i^2}$$

For a linear model $y = mx$ (like Ohm's Law where $V = RI$), we can write this in matrix form.

### Matrix Formulation

Define the **weight matrix** $\mathbf{W}$ as a diagonal matrix with weights $w_i = 1/\sigma_i^2$:

$$\mathbf{W} = \begin{bmatrix}
1/\sigma_1^2 & 0 & \cdots & 0 \\
0 & 1/\sigma_2^2 & \cdots & 0 \\
\vdots & \vdots & \ddots & \vdots \\
0 & 0 & \cdots & 1/\sigma_N^2
\end{bmatrix}$$

For the linear model $\mathbf{y} = \mathbf{A}\mathbf{x}$ where $\mathbf{A}$ is the design matrix (in our case, just the column of current values), the **weighted least squares solution** is:

$$\mathbf{x} = (\mathbf{A}^T \mathbf{W} \mathbf{A})^{-1} \mathbf{A}^T \mathbf{W} \mathbf{y}$$

For the simple case of fitting $V = R \cdot I$ (a line through the origin), this simplifies to:

$$R = \frac{\sum_i w_i I_i V_i}{\sum_i w_i I_i^2} = \frac{\sum_i \frac{I_i V_i}{\sigma_i^2}}{\sum_i \frac{I_i^2}{\sigma_i^2}}$$

### Uncertainty in the Fitted Parameter

The uncertainty in the fitted resistance is given by:

$$\sigma_R = \sqrt{(\mathbf{A}^T \mathbf{W} \mathbf{A})^{-1}} = \sqrt{\frac{1}{\sum_i \frac{I_i^2}{\sigma_i^2}}}$$

### Your Task

Write a function `weighted_least_squares_fit(filename, show_plot=False)` that:
1. Reads data from a CSV file containing columns: 'current' (I), 'voltage' (V), and 'uncertainty' (σ)
2. Performs weighted least squares fitting to find the resistance R
3. Calculates the uncertainty in R
4. Creates a plot showing:
   - Original data points with error bars
   - The fitted line V = R·I
   - A shaded region showing the uncertainty in the fit
5. Returns the resistance, its uncertainty, and the figure object

**Requirements:**
- Read CSV data using pandas
- Compute weights as $w_i = 1/\sigma_i^2$
- Calculate fitted resistance using the weighted least squares formula
- Calculate uncertainty in resistance
- Plot data with error bars using `plt.errorbar()`
- Plot the fitted line
- Include appropriate labels, title, legend, and grid

**Parameters:**
- `filename`: str, path to CSV file with 'current', 'voltage', 'uncertainty' columns
- `show_plot`: bool, default False. If True, call `plt.show()`

**Returns:**
- `(R, sigma_R, fig)`: tuple containing:
  - `R`: float, fitted resistance in Ohms
  - `sigma_R`: float, uncertainty in resistance
  - `fig`: matplotlib figure object

**Hints:**
- Use `plt.errorbar(x, y, yerr=uncertainties, fmt='o')` to plot data with error bars
- Generate a smooth line for the fit using many current values
- The shaded uncertainty region can be plotted with `plt.fill_between()`

In [ ]:
def weighted_least_squares_fit(filename, show_plot=False):
    
    # Read the data
    data = pd.read_csv(filename)
    I = ...
    V = ...
    sigma = ...
    
    # Compute weights
    
    #Find WLS R and uncertainty in R
    
    # Create plot
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Plot data with error bars
    
    # Plot fitted line
    
    if show_plot:
        plt.show()
    
    return R, sigma_R, fig

In [ ]:
# Test the weighted least squares fitting function

# Perform the fit
R_fit, sigma_R_fit, fig = weighted_least_squares_fit('resistance_data.csv', show_plot=True)

In [ ]:
grader.check("q3")

## Required disclosure of use of AI technology

Please indicate whether you used AI to complete this homework. If you did, explain how you used it in the python cell below, as a comment.

In [ ]:
"""
# write ai disclosure here:

"""

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit.

Upload the .zip file to Gradescope!

In [ ]:
grader.export(pdf=False, force_save=True, run_tests=True)